# Graph RAG: 图增强检索增强生成

在这个笔记本中，我实现了 Graph RAG - 一种通过将知识组织为连接图而不是平面文档集合来增强传统 RAG 系统的技术。这使得系统能够导航相关概念并检索比标准向量相似性方法更具上下文相关性的信息。

Graph RAG 的主要优势

- 保留信息片段之间的关系
- 能够通过连接的概念进行遍历以找到相关上下文
- 改善对复杂、多部分查询的处理
- 通过可视化知识路径提供更好的可解释性

## 设置环境
我们首先导入必要的库。

In [1]:
import os
import numpy as np
import json
import fitz  # PyMuPDF
from openai import OpenAI
from typing import List, Dict, Tuple, Any
import networkx as nx
import matplotlib.pyplot as plt
import heapq
from collections import defaultdict
import re
from PIL import Image
import io

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 文档处理函数

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本内容。
    
    Args:
        pdf_path (str): PDF 文件路径
        
    Returns:
        str: 提取的文本内容
    """
    print(f"正在从 {pdf_path} 提取文本...")  # 打印正在处理的 PDF 路径
    pdf_document = fitz.open(pdf_path)  # 使用 PyMuPDF 打开 PDF 文件
    text = ""  # 初始化空字符串来存储提取的文本
    
    # 遍历 PDF 中的每一页
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]  # 获取页面对象
        text += page.get_text()  # 从页面提取文本并追加到文本字符串中
    
    return text  # 返回提取的文本内容

In [4]:
def chunk_text(text, chunk_size=1000, overlap=200):
    """
    将文本分割为重叠的块。
    
    Args:
        text (str): 要分块的输入文本
        chunk_size (int): 每个块的字符大小
        overlap (int): 块之间的重叠字符数
        
    Returns:
        List[Dict]: 包含元数据的块列表
    """
    chunks = []  # 初始化空列表来存储块
    
    # 以 (chunk_size - overlap) 的步长遍历文本
    for i in range(0, len(text), chunk_size - overlap):
        # 从当前位置提取一个文本块
        chunk_text = text[i:i + chunk_size]
        
        # 确保不添加空块
        if chunk_text:
            # 将块及其元数据追加到列表中
            chunks.append({
                "text": chunk_text,  # 文本块
                "index": len(chunks),  # 块的索引
                "start_pos": i,  # 块在原始文本中的起始位置
                "end_pos": i + len(chunk_text)  # 块在原始文本中的结束位置
            })
    
    # 打印创建的块数量
    print(f"创建了 {len(chunks)} 个文本块")
    
    return chunks  # 返回块列表

## 创建嵌入向量

In [5]:
def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量。
    
    Args:
        texts (List[str]): 输入文本
        model (str): 嵌入模型名称
        
    Returns:
        List[List[float]]: 嵌入向量
    """
    # 处理空输入
    if not texts:
        return []
        
    # 如果需要，分批处理（OpenAI API 限制）
    batch_size = 100
    all_embeddings = []
    
    # 分批遍历输入文本
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 获取当前批次的文本
        
        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到列表中
    
    return all_embeddings  # 返回所有嵌入向量

## 知识图谱构建

In [6]:
def extract_concepts(text):
    """
    使用 OpenAI 的 API 从文本中提取关键概念。
    
    Args:
        text (str): 要提取概念的文本
        
    Returns:
        List[str]: 概念列表
    """
    # 指导模型执行任务的系统消息
    system_message = """从提供的文本中提取关键概念和实体。
只返回 5-10 个在此文本中最重要的关键术语、实体或概念的列表。
将您的响应格式化为字符串的 JSON 数组。"""

    # 向 OpenAI API 发出请求
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"从以下内容中提取关键概念：\n\n{text[:3000]}"}  # API 限制
        ],
        temperature=0.0,
        response_format={"type": "json_object"}
    )
    
    try:
        # 从响应中解析概念
        concepts_json = json.loads(response.choices[0].message.content)
        concepts = concepts_json.get("concepts", [])
        if not concepts and "concepts" not in concepts_json:
            # 尝试获取响应中的任何数组
            for key, value in concepts_json.items():
                if isinstance(value, list):
                    concepts = value
                    break
        return concepts
    except (json.JSONDecodeError, AttributeError):
        # 如果 JSON 解析失败的回退方案
        content = response.choices[0].message.content
        # 尝试提取任何看起来像列表的内容
        matches = re.findall(r'\[(.*?)\]', content, re.DOTALL)
        if matches:
            items = re.findall(r'"([^"]*)"', matches[0])
            return items
        return []

In [7]:
def build_knowledge_graph(chunks):
    """
    从文本块构建知识图谱。
    
    Args:
        chunks (List[Dict]): 包含元数据的文本块列表
        
    Returns:
        Tuple[nx.Graph, List[np.ndarray]]: 知识图谱和块嵌入向量
    """
    print("正在构建知识图谱...")
    
    # 创建图
    graph = nx.Graph()
    
    # 提取块文本
    texts = [chunk["text"] for chunk in chunks]
    
    # 为所有块创建嵌入向量
    print("正在为块创建嵌入向量...")
    embeddings = create_embeddings(texts)
    
    # 向图中添加节点
    print("正在向图中添加节点...")
    for i, chunk in enumerate(chunks):
        # 从块中提取概念
        print(f"正在为块 {i+1}/{len(chunks)} 提取概念...")
        concepts = extract_concepts(chunk["text"])
        
        # 添加带有属性的节点
        graph.add_node(i, 
                      text=chunk["text"], 
                      concepts=concepts,
                      embedding=embeddings[i])
    
    # 基于共享概念连接节点
    print("正在创建节点之间的边...")
    for i in range(len(chunks)):
        node_concepts = set(graph.nodes[i]["concepts"])
        
        for j in range(i + 1, len(chunks)):
            # 计算概念重叠
            other_concepts = set(graph.nodes[j]["concepts"])
            shared_concepts = node_concepts.intersection(other_concepts)
            
            # 如果它们共享概念，添加一条边
            if shared_concepts:
                # 使用嵌入向量计算语义相似性
                similarity = np.dot(embeddings[i], embeddings[j]) / (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
                
                # 基于概念重叠和语义相似性计算边权重
                concept_score = len(shared_concepts) / min(len(node_concepts), len(other_concepts))
                edge_weight = 0.7 * similarity + 0.3 * concept_score
                
                # 只添加具有显著关系的边
                if edge_weight > 0.6:
                    graph.add_edge(i, j, 
                                  weight=edge_weight,
                                  similarity=similarity,
                                  shared_concepts=list(shared_concepts))
    
    print(f"知识图谱构建完成，包含 {graph.number_of_nodes()} 个节点和 {graph.number_of_edges()} 条边")
    return graph, embeddings

## 图遍历和查询处理

In [8]:
def traverse_graph(query, graph, embeddings, top_k=5, max_depth=3):
    """
    遍历知识图谱以找到查询的相关信息。
    
    Args:
        query (str): 用户的问题
        graph (nx.Graph): 知识图谱
        embeddings (List): 节点嵌入向量列表
        top_k (int): 要考虑的初始节点数量
        max_depth (int): 最大遍历深度
        
    Returns:
        List[Dict]: 从图遍历中获得的相关信息
    """
    print(f"正在为查询遍历图：{query}")
    
    # 获取查询嵌入向量
    query_embedding = create_embeddings(query)
    
    # 计算查询与所有节点之间的相似性
    similarities = []
    for i, node_embedding in enumerate(embeddings):
        similarity = np.dot(query_embedding, node_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(node_embedding))
        similarities.append((i, similarity))
    
    # 按相似性排序（降序）
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # 获取前 k 个最相似的节点作为起始点
    starting_nodes = [node for node, _ in similarities[:top_k]]
    print(f"从 {len(starting_nodes)} 个节点开始遍历")
    
    # 初始化遍历
    visited = set()  # 用于跟踪已访问节点的集合
    traversal_path = []  # 存储遍历路径的列表
    results = []  # 存储结果的列表
    
    # 使用优先队列进行遍历
    queue = []
    for node in starting_nodes:
        heapq.heappush(queue, (-similarities[node][1], node))  # 负数用于最大堆
    
    # 使用带优先级的修改广度优先搜索遍历图
    while queue and len(results) < (top_k * 3):  # 将结果限制为 top_k * 3
        _, node = heapq.heappop(queue)
        
        if node in visited:
            continue
        
        # 标记为已访问
        visited.add(node)
        traversal_path.append(node)
        
        # 将当前节点的文本添加到结果中
        results.append({
            "text": graph.nodes[node]["text"],
            "concepts": graph.nodes[node]["concepts"],
            "node_id": node
        })
        
        # 如果我们还没有达到最大深度，探索邻居
        if len(traversal_path) < max_depth:
            neighbors = [(neighbor, graph[node][neighbor]["weight"]) 
                        for neighbor in graph.neighbors(node)
                        if neighbor not in visited]
            
            # 基于边权重将邻居添加到队列中
            for neighbor, weight in sorted(neighbors, key=lambda x: x[1], reverse=True):
                heapq.heappush(queue, (-weight, neighbor))
    
    print(f"图遍历找到了 {len(results)} 个相关块")
    return results, traversal_path

## 响应生成

In [9]:
def generate_response(query, context_chunks):
    """
    使用检索到的上下文生成响应。
    
    Args:
        query (str): 用户的问题
        context_chunks (List[Dict]): 从图遍历中获得的相关块
        
    Returns:
        str: 生成的响应
    """
    # 从上下文中的每个块提取文本
    context_texts = [chunk["text"] for chunk in context_chunks]
    
    # 将提取的文本合并为单个上下文字符串，用 "---" 分隔
    combined_context = "\n\n---\n\n".join(context_texts)
    
    # 定义上下文的最大允许长度（OpenAI 限制）
    max_context = 14000
    
    # 如果合并的上下文超过最大长度，则截断
    if len(combined_context) > max_context:
        combined_context = combined_context[:max_context] + "... [已截断]"
    
    # 定义指导 AI 助手的系统消息
    system_message = """您是一个有用的 AI 助手。根据提供的上下文回答用户的问题。
如果信息不在上下文中，请说明。在可能的情况下，在您的答案中引用上下文的特定部分。"""

    # 使用 OpenAI API 生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_message},  # 指导助手的系统消息
            {"role": "user", "content": f"上下文：\n{combined_context}\n\n问题：{query}"}  # 包含上下文和查询的用户消息
        ],
        temperature=0.2  # 设置响应生成的温度
    )
    
    # 返回生成的响应内容
    return response.choices[0].message.content

## 可视化

In [10]:
def visualize_graph_traversal(graph, traversal_path):
    """
    可视化知识图谱和遍历路径。
    
    Args:
        graph (nx.Graph): 知识图谱
        traversal_path (List): 按遍历顺序的节点列表
    """
    plt.figure(figsize=(12, 10))  # 设置图形大小
    
    # 定义节点颜色，默认为浅蓝色
    node_color = ['lightblue'] * graph.number_of_nodes()
    
    # 将遍历路径节点高亮为浅绿色
    for node in traversal_path:
        node_color[node] = 'lightgreen'
    
    # 将起始节点高亮为绿色，结束节点高亮为红色
    if traversal_path:
        node_color[traversal_path[0]] = 'green'
        node_color[traversal_path[-1]] = 'red'
    
    # 使用弹簧布局为所有节点创建位置
    pos = nx.spring_layout(graph, k=0.5, iterations=50, seed=42)
    
    # 绘制图节点
    nx.draw_networkx_nodes(graph, pos, node_color=node_color, node_size=500, alpha=0.8)
    
    # 绘制边，宽度与权重成比例
    for u, v, data in graph.edges(data=True):
        weight = data.get('weight', 1.0)
        nx.draw_networkx_edges(graph, pos, edgelist=[(u, v)], width=weight*2, alpha=0.6)
    
    # 用红色虚线绘制遍历路径
    traversal_edges = [(traversal_path[i], traversal_path[i+1]) 
                      for i in range(len(traversal_path)-1)]
    
    nx.draw_networkx_edges(graph, pos, edgelist=traversal_edges, 
                          width=3, alpha=0.8, edge_color='red', 
                          style='dashed', arrows=True)
    
    # 为每个节点添加第一个概念的标签
    labels = {}
    for node in graph.nodes():
        concepts = graph.nodes[node]['concepts']
        label = concepts[0] if concepts else f"节点 {node}"
        labels[node] = f"{node}: {label}"
    
    nx.draw_networkx_labels(graph, pos, labels=labels, font_size=8)
    
    plt.title("带遍历路径的知识图谱")  # 设置图标题
    plt.axis('off')  # 关闭坐标轴
    plt.tight_layout()  # 调整布局
    plt.show()  # 显示图

## 完整的 Graph RAG 流水线

In [11]:
def graph_rag_pipeline(pdf_path, query, chunk_size=1000, chunk_overlap=200, top_k=3):
    """
    从文档到答案的完整 Graph RAG 流水线。
    
    Args:
        pdf_path (str): PDF 文档路径
        query (str): 用户的问题
        chunk_size (int): 文本块大小
        chunk_overlap (int): 块之间的重叠
        top_k (int): 遍历时要考虑的顶级节点数量
        
    Returns:
        Dict: 包括答案和图可视化数据的结果
    """
    # 从 PDF 文档中提取文本
    text = extract_text_from_pdf(pdf_path)
    
    # 将提取的文本分割为重叠的块
    chunks = chunk_text(text, chunk_size, chunk_overlap)
    
    # 从文本块构建知识图谱
    graph, embeddings = build_knowledge_graph(chunks)
    
    # 遍历知识图谱以找到查询的相关信息
    relevant_chunks, traversal_path = traverse_graph(query, graph, embeddings, top_k)
    
    # 基于查询和相关块生成响应
    response = generate_response(query, relevant_chunks)
    
    # 可视化图遍历路径
    visualize_graph_traversal(graph, traversal_path)
    
    # 返回查询、响应、相关块、遍历路径和图
    return {
        "query": query,
        "response": response,
        "relevant_chunks": relevant_chunks,
        "traversal_path": traversal_path,
        "graph": graph
    }

## 评估函数

In [12]:
def evaluate_graph_rag(pdf_path, test_queries, reference_answers=None):
    """
    在多个测试查询上评估 Graph RAG。
    
    Args:
        pdf_path (str): PDF 文档路径
        test_queries (List[str]): 测试查询列表
        reference_answers (List[str], optional): 用于比较的参考答案
        
    Returns:
        Dict: 评估结果
    """
    # 从 PDF 中提取文本
    text = extract_text_from_pdf(pdf_path)
    
    # 将文本分割为块
    chunks = chunk_text(text)
    
    # 构建知识图谱（对所有查询只做一次）
    graph, embeddings = build_knowledge_graph(chunks)
    
    results = []
    
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 评估查询 {i+1}/{len(test_queries)} ===")
        print(f"查询：{query}")
        
        # 遍历图以找到相关信息
        relevant_chunks, traversal_path = traverse_graph(query, graph, embeddings)
        
        # 生成响应
        response = generate_response(query, relevant_chunks)
        
        # 如果有参考答案，与之比较
        reference = None
        comparison = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
            comparison = compare_with_reference(response, reference, query)
        
        # 为当前查询追加结果
        results.append({
            "query": query,
            "response": response,
            "reference_answer": reference,
            "comparison": comparison,
            "traversal_path_length": len(traversal_path),
            "relevant_chunks_count": len(relevant_chunks)
        })
        
        # 显示结果
        print(f"\n响应：{response}\n")
        if comparison:
            print(f"比较：{comparison}\n")
    
    # 返回评估结果和图统计信息
    return {
        "results": results,
        "graph_stats": {
            "nodes": graph.number_of_nodes(),
            "edges": graph.number_of_edges(),
            "avg_degree": sum(dict(graph.degree()).values()) / graph.number_of_nodes()
        }
    }

In [13]:
def compare_with_reference(response, reference, query):
    """
    将生成的响应与参考答案进行比较。
    
    Args:
        response (str): 生成的响应
        reference (str): 参考答案
        query (str): 原始查询
        
    Returns:
        str: 比较分析
    """
    # 指导模型如何比较响应的系统消息
    system_message = """比较 AI 生成的响应与参考答案。
基于以下方面进行评估：正确性、完整性和与查询的相关性。
提供简要分析（2-3 句话），说明生成的响应与参考答案的匹配程度。"""

    # 构建包含查询、AI 生成响应和参考答案的提示
    prompt = f"""
查询：{query}

AI 生成的响应：
{response}

参考答案：
{reference}

AI 响应与参考答案的匹配程度如何？
"""

    # 向 OpenAI API 发出请求以生成比较分析
    comparison = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_message},  # 指导助手的系统消息
            {"role": "user", "content": prompt}  # 包含提示的用户消息
        ],
        temperature=0.0  # 设置响应生成的温度
    )
    
    # 返回生成的比较分析
    return comparison.choices[0].message.content

## 在示例 PDF 文档上评估 Graph RAG

In [14]:
# 示例使用
if __name__ == "__main__":
    # 定义 PDF 路径和查询
    pdf_path = "data/AI_Information.pdf"
    query = "Transformer 在自然语言处理中的关键应用有哪些？"
    
    # 运行 Graph RAG 流水线
    result = graph_rag_pipeline(pdf_path, query)
    
    print(f"查询：{result['query']}")
    print(f"响应：{result['response']}")
    print(f"遍历路径长度：{len(result['traversal_path'])}")
    print(f"相关块数量：{len(result['relevant_chunks'])}")

## 总结

Graph RAG 通过以下方式增强了传统的 RAG 系统：

1. **结构化知识表示**：将文档组织为连接的概念图，而不是独立的文本块
2. **关系感知检索**：利用概念之间的关系来找到更相关的信息
3. **智能遍历**：通过图结构导航以发现相关但可能不直接相似的信息
4. **可解释性**：提供清晰的推理路径，显示信息是如何连接的

这种方法特别适用于：
- 复杂的多步推理查询
- 需要综合多个相关概念的问题
- 要求高可解释性的应用场景
- 处理具有丰富内部关系的文档集合

Graph RAG 代表了检索增强生成技术的重要进步，为构建更智能、更具上下文感知能力的问答系统提供了强大的框架。